In [ ]:
Ensemble 

1. Doc Classification
-> image Recognition

Checks    W2's Apps


2.Indvividual Doc OCR
AWS TextExtract 

Comprehend 


In [1]:
import pandas as pd
data = {'Sleeve': [1, 2, 3],
        'Quality': [4, 4, 2]}

df = pd.DataFrame (data, columns = ['Sleeve','Quality'])
print(df)

   Sleeve  Quality
0       1        4
1       2        4
2       3        2


In [2]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(df)
print(similarity)

[[1.         0.97618706 0.73994007]
 [0.97618706 1.         0.86824314]
 [0.73994007 0.86824314 1.        ]]


Two vectors in Cartesian space are considered to be equal if

Both have the same length
Both are in the same direction and are parallel to each other

In [ ]:
y = f(x) + b

if at any moment you have a linear relationship

w = g(f(x))

y = w(x|)




Cosine similarity is a metric used to determine how similar the documents are irrespective of their size.

In [4]:
# Define the documents
doc_trump = "Mr. Trump became president after winning the political election. Though he lost the support of some republican friends, Trump is friends with President Putin"

doc_election = "President Trump says Putin had no political interference is the election outcome. He says it was a witchhunt by political parties. He claimed President Putin is a friend who had nothing to do with the election"

doc_putin = "Post elections, Vladimir Putin became President of Russia. President Putin had served as the Prime Minister earlier in his political career"

documents = [doc_trump, doc_election, doc_putin]


In [5]:
# Scikit Learn
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Create the Document Term Matrix
count_vectorizer = CountVectorizer(stop_words='english')
count_vectorizer = CountVectorizer()
sparse_matrix = count_vectorizer.fit_transform(documents)

# OPTIONAL: Convert Sparse Matrix to Pandas Dataframe if you want to see the word frequencies.
doc_term_matrix = sparse_matrix.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=count_vectorizer.get_feature_names(), 
                  index=['doc_trump', 'doc_election', 'doc_putin'])
df

,after,as,became,by,career,claimed,do,earlier,election,elections,...,the,though,to,trump,vladimir,was,who,winning,witchhunt,with
doc_trump,1,0,1,0,0,0,0,0,1,0,...,2,1,0,2,0,0,0,1,0,1
doc_election,0,0,0,1,0,1,1,0,2,0,...,2,0,1,1,0,1,1,0,1,1
doc_putin,0,1,1,0,1,0,0,1,0,1,...,1,0,0,0,1,0,0,0,0,0


In [6]:
from sklearn.metrics.pairwise import cosine_similarity
print(cosine_similarity(df, df))
#> [[ 1.          0.48927489  0.37139068]
#>  [ 0.48927489  1.          0.38829014]
#>  [ 0.37139068  0.38829014  1.        ]]

[[1.         0.51480485 0.38890873]
 [0.51480485 1.         0.38829014]
 [0.38890873 0.38829014 1.        ]]


In [7]:
# Define the documents
doc_soup = "Soup is a primarily liquid food, generally served warm or hot (but may be cool or cold), that is made by combining ingredients of meat or vegetables with stock, juice, water, or another liquid. "

doc_noodles = "Noodles are a staple food in many cultures. They are made from unleavened dough which is stretched, extruded, or rolled flat and cut into one of a variety of shapes."

doc_dosa = "Dosa is a type of pancake from the Indian subcontinent, made from a fermented batter. It is somewhat similar to a crepe in appearance. Its main ingredients are rice and black gram."

documents = [doc_trump, doc_election, doc_putin, doc_soup, doc_noodles, doc_dosa]

In [9]:
!pip install gensim

  Created wheel for smart-open: filename=smart_open-4.1.0-py3-none-any.whl size=106208 sha256=a30e389293fe47682610afd06a55ba33de68f821488b67ad89380a3d8183c9c4
  Stored in directory: c:\users\zebfred\appdata\local\pip\cache\wheels\74\33\8e\37e22f50ce94856f37b3e23a93c648c194aa8d5a546083d09a
Successfully built smart-open


In [10]:
import gensim
# upgrade gensim if you can't import softcossim
from gensim.matutils import softcossim 
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
print(gensim.__version__)
#> '3.6.0'

# Download the FastText model
fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')

3.8.3
[=================================================-] 99.8% 956.2/958.4MB downloaded

In [11]:
# Prepare a dictionary and a corpus.
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in documents])

# Prepare the similarity matrix
similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)

# Convert the sentences into bag-of-words vectors.
sent_1 = dictionary.doc2bow(simple_preprocess(doc_trump))
sent_2 = dictionary.doc2bow(simple_preprocess(doc_election))
sent_3 = dictionary.doc2bow(simple_preprocess(doc_putin))
sent_4 = dictionary.doc2bow(simple_preprocess(doc_soup))
sent_5 = dictionary.doc2bow(simple_preprocess(doc_noodles))
sent_6 = dictionary.doc2bow(simple_preprocess(doc_dosa))

sentences = [sent_1, sent_2, sent_3, sent_4, sent_5, sent_6]

<ipython-input-11-70ef5acc1290>:5: DeprecationWarning: Call to deprecated `similarity_matrix` (Method will be removed in 4.0.0, use gensim.models.keyedvectors.WordEmbeddingSimilarityIndex instead).
  similarity_matrix = fasttext_model300.similarity_matrix(dictionary, tfidf=None, threshold=0.0, exponent=2.0, nonzero_limit=100)


In [ ]:
# Compute soft cosine similarity
print(softcossim(sent_1, sent_2, similarity_matrix))
#> 0.567228632589

In [ ]:
import numpy as np
import pandas as pd

def create_soft_cossim_matrix(sentences):
    len_array = np.arange(len(sentences))
    xx, yy = np.meshgrid(len_array, len_array)
    cossim_mat = pd.DataFrame([[round(softcossim(sentences[i],sentences[j], similarity_matrix) ,2) for i, j in zip(x,y)] for y, x in zip(xx, yy)])
    return cossim_mat

soft_cosine_similarity_matrix(sentences

In [ ]:
# 1. No Multicolinearity 
# 2. Linear relationship
# 3. Homoscedasticity
# 4. Normality 

In [ ]:
#need to build my own sklearn
y = mx + b

y = wx + b 

weights 
bias

In [ ]:
Regression Task

1.No Multicolinearity

x0 , x1 , x2 , x3 , x4 , x5

1. correlation matrix 



In [ ]:
#dive deeper into making your own knn with 
# 

# what the different statical correlations
# coding problem daily
# doing codesignal coding problems
# studying anki cards

#email 
#fix presentation code 


In [ ]:
1 No multicolinearity

continous values linear regression

multicolinearity test

In [ ]:
meat vs grass 

basic 

least powerful tool

Factory , lots of different procedure 

Bayes -multinomial , gauss,  Naive
multivariate - multivariable 
a dicrete distrubation

Building a useful app
Using machine learning 
building game dev 
Using family of C , java, udacity




In [ ]:
Start Simple 
1. Pick simplist model
2. get metrics
3. if metrics = threshold:STOP
4. Else complexitry = 1
Goto 1

Base Classify

In [ ]:
Basic
Young Text and classification
Stories 

drawing 
art 
working out 


In [ ]:
20 basic models we can make ensembles 

